In [21]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.pipeline import make_pipeline
from sklearn.compose import make_column_selector, make_column_transformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import PolynomialFeatures

In [22]:
df=pd.read_excel('FullData.xls')
df.shape

(45892, 22)

In [77]:
#df.isnull().sum().sort_values(ascending=False)
#df.head(10)

In [24]:
df=df.dropna(subset=['cumul_cmd']) 
df.shape

(45884, 22)

In [25]:
y_train= df['cumul_cmd']
X_train= df.drop('cumul_cmd', axis=1)

In [26]:
numerical_features=X_train.select_dtypes(include=['int64']).columns
categorical_features=df.select_dtypes(include='object').columns

In [27]:
numerical_pipeline = make_pipeline(SimpleImputer(strategy='mean'), StandardScaler())
categorical_pipeline = make_pipeline(SimpleImputer(strategy='most_frequent'), OneHotEncoder())

In [28]:
preprocessor = make_column_transformer((numerical_pipeline, numerical_features),
                                   (categorical_pipeline, categorical_features))


In [29]:
model = make_pipeline(preprocessor, RandomForestRegressor())

In [30]:
model.fit(X_train, y_train)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('pipeline-1',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer()),
                                                                  ('standardscaler',
                                                                   StandardScaler())]),
                                                  Index(['cumul_visit', 'moy_satisf', 'visit_period', 'frequency'], dtype='object')),
                                                 ('pipeline-2',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehotencoder',
                                                                   OneHotEncoder(

In [75]:
model.score(X_train, y_train)

0.9122211926901054

In [76]:
import pickle
pickle.dump(model, open('RFmodel', 'wb'))

In [53]:
x=np.array([np.datetime64('2021-12-27'),'Parapharmacie','Parapharmacie','B','P','PARAPHARMACIE','Medenine 2','MIDOUN JERBA'
            ,'ORTHOKARE DENT',np.nan
,0,0,0,1,4,np.datetime64('2021-12-27'),np.datetime64('2021-12-27'),np.datetime64('2021-12-27'),0,0,0]).reshape(1,21)

In [54]:
X= pd.DataFrame(x,columns=X_train.columns)

,visit_date,first_name,last_name,potencial,activity,speciality,sector,locality,product,general_note,...,cumul_sale,cumul_pres,cumul_visit,moy_satisf,first_visit,previous_visit,last_visit,visit_period,frequency,recency
0,2021-12-27,Parapharmacie,Parapharmacie,B,P,PARAPHARMACIE,Medenine 2,MIDOUN JERBA,ORTHOKARE DENT,NaN,...,0,0,1,4,2021-12-27,2021-12-27,2021-12-27,0,0,0


In [55]:
model.predict(X)

array([0.])

In [73]:
def predict_cmd(model,visit_date, first_name, last_name, potencial, activity,
       speciality, sector, locality, product, general_note,
       cumul_ech, cumul_sale, cumul_pres, cumul_visit,
       moy_satisf, first_visit, previous_visit, last_visit,
       visit_period, frequency, recency ):
    y = np.array([visit_date, first_name, last_name, potencial, activity,
       speciality, sector, locality, product, general_note,
       cumul_ech,  cumul_sale, cumul_pres, cumul_visit,
       moy_satisf, first_visit, previous_visit, last_visit,
       visit_period, frequency, recency]).reshape(1, 21)
    Y= pd.DataFrame(y,columns=X_train.columns)
    print(model.predict(Y))


In [74]:
predict_cmd(model,np.datetime64('2021-12-27'),'Parapharmacie','Parapharmacie','B','P','PARAPHARMACIE','Medenine 2','MIDOUN JERBA'
            ,'ORTHOKARE DENT',np.nan,0,0,0,1,4,np.datetime64('2021-12-27'),
            np.datetime64('2021-12-27'),np.datetime64('2021-12-27'),0,0,0)

[0.]
